In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.getcwd()
df = pd.read_csv("csv/traj.csv")

In [ ]:


# ==============================
# 튜닝 가는 세팅
# ==============================
SETTLING_BAND = 0.05          # 5% band
STEP_IGNORE_SEC = 0.10        # step 직후 0.1초는 지표 계산에서 제외(권장)
STEADY_WINDOW_SEC = 1.0       # steady-state error 계산 구간(마지막 1초 평균)
MIN_STEP_AMPLITUDE = 1e-6     # ref 변화가 이 값보다 작으면 step으로 무시.

# ==============================
# 헬퍼 함수 정의
# ==============================
def find_step_segments(df: pd.DataFrame, ref_cols):
    """
    ref_cols의 값 변화로 구간을 나눔.
    return: list of (start_idx, end_idx) inclusive
    """
    ref_mat = df[ref_cols].to_numpy()
    # change occurs when any ref component changes from previous sample
    change = np.any(np.abs(ref_mat[1:] - ref_mat[:-1]) > MIN_STEP_AMPLITUDE, axis=1)
    change_idx = np.where(change)[0] + 1  # indices where new segment starts
    starts = np.r_[0, change_idx]
    ends = np.r_[change_idx - 1, len(df) - 1]
    return list(zip(starts, ends))

def settling_time(t, err, band_abs):
    """
    band_abs: 절대 밴드(예: 0.05*|step_amp|)
    반환: settling time (초), 없으면 NaN
    """
    # 밴드가 0 이하(즉, 스탭이 없음)면 "이미 정착됨"으로 간주
    if band_abs <= 0:
        return np.nan
        return 0.0

    within = np.abs(err) <= band_abs
    if not np.any(within):
        return np.nan

    # "한 번 들어간 뒤 끝까지 계속 within"인 최초 인덱스 찾기
    # suffix_all[i] = within[i] & within[i+1] & ... & within[end]
    suffix_all = np.logical_and.accumulate(within[::-1])[::-1]
    idx = np.where(suffix_all)[0]
    if len(idx) == 0:
        return np.nan
    return float(t[idx[0]] - t[0])

def overshoot_percent(y, y0, yf):
    """
    step 응답 overshoot(%)
    y0: step 직전 값(또는 구간 시작값), yf: 최종 ref.
    양/음 step 모두 처리.
    """
    amp = yf - y0
    if abs(amp) <= MIN_STEP_AMPLITUDE:
        return np.nan

    if amp > 0:
        peak = np.max(y)
        os = (peak - yf) / abs(amp) * 100.0
    else:
        trough = np.min(y)
        os = (yf - trough) / abs(amp) * 100.0

    return float(max(os, 0.0))  # 음수 overshoot는 0으로 처리(원하면 그대로 반환 가능)

def rmse(err):
    return float(np.sqrt(np.mean(err**2))) if len(err) else np.nan

# ==============================
# Main
# ==============================

required = {"t", "px", "py", "pz", "refx", "refy", "refz"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV에 필요한 컬럼이 없습니다: {missing}")

t_all = df["t"].to_numpy()

segments = find_step_segments(df, ["refx", "refy", "refz"])

rows = []
for seg_i, (i0, i1) in enumerate(segments):
    seg = df.iloc[i0:i1+1].copy()
    t = seg["t"].to_numpy()

    # 구간의 "목표 ref"는 구간 내에서 상수라고 가정 (SITL ref step 로깅 기준)
    refx = float(seg["refx"].iloc[-1])
    refy = float(seg["refy"].iloc[-1])
    refz = float(seg["refz"].iloc[-1])

    # step 직후 일부 구간 제외(지표가 더 안정적이됨)
    t0 = t[0]
    mask_eval = t >= (t0 + STEP_IGNORE_SEC)
    seg_eval = seg.loc[mask_eval]
    t_eval = seg_eval["t"].to_numpy()

    # steady-state 구간(끝에서 STEADY_WINDOW_SEC)
    t_end = t[-1]
    mask_steady = t >= (t_end - STEADY_WINDOW_SEC)
    seg_steady = seg.loc[mask_steady]

    # 직전 구간의 마지막 상태를 step 이전 값(y0)으로 사용.
    if seg_i == 0:
        # 첫 구간이면 시작값을 y0로 둠(혹은 ref 변화 없으면 의미 없음)
        y0x = float(seg["px"].iloc[0])
        y0y = float(seg["py"].iloc[0])
        y0z = float(seg["pz"].iloc[0])
    else:
        prev_end = df.iloc[segments[seg_i-1][1]]
        y0x = float(prev_end["px"])
        y0y = float(prev_end["py"])
        y0z = float(prev_end["pz"])

    # 각 축별 계산
    for axis, y_col, r_final, y0 in [
        ("x", "px", refx, y0x),
        ("y", "py", refy, y0y),
        ("z", "pz", refz, y0z),
    ]:
        y = seg[y_col].to_numpy()
        y_eval = seg_eval[y_col].to_numpy()

        # 에러 (evaluation 구간)
        err_eval = y_eval - r_final if len(y_eval) else np.array([])
        err_all = y - r_final

        step_amp = r_final - y0
        band_abs = SETTLING_BAND * abs(step_amp)
        
        # [수정] 스텝 크기가 10cm 미만이면 오버슈트, satteling  time 계산 안 함.
        if abs(step_amp) < 0.1:
            os_pct = 0.0
            st = 0.0
        else:
            # 10cm 이상 움직일 때만 진짜 오버슈트 계산.
            os_pct = overshoot_percent(y_eval if len(y_eval) else y, y0, r_final)
        # os_pct = overshoot_percent(y_eval if len(y_eval) else y, y0, r_final)
        st = settling_time(t_eval if len(t_eval) else t, err_eval if len(err_eval) else err_all, band_abs)

        # steady-state error: 마지막 STEADY_WINDOW_SEC 평균 오차.
        steady_err = float((seg_steady[y_col].to_numpy() - r_final).mean()) if len(seg_steady) else np.nan

        # RMSE: evaluation 구간 RMSE.
        rmse_val = rmse(err_eval) if len(err_eval) else rmse(err_all)

        rows.append({
            "segment": seg_i,
            "t_start": float(t[0]),
            "t_end": float(t[-1]),
            "axis": axis,
            "ref_final": r_final,
            "y0": y0,
            "step_amp": step_amp,
            "settling_band": SETTLING_BAND,
            "overshoot_pct": os_pct,
            "settling_time_s": st,
            "steady_state_error": steady_err,
            "rmse": rmse_val,
        })
# 보기 좋게 정렬 및 저장
metrics = pd.DataFrame(rows)
metrics = metrics.sort_values(["segment", "axis"]).reset_index(drop=True)
out_path = "csv/metrics_summary.csv"
metrics.to_csv(out_path, index=False)
# 콘솔 출력(요약)
print("=== Metrics (per segment, per axis) ===")
print(metrics[[
    "segment","t_start","t_end","axis","ref_final","step_amp",
    "overshoot_pct","settling_time_s","steady_state_error","rmse"
]].to_string(index=False))
print(f"\nSaved: {out_path}")

=== Metrics (per segment, per axis) ===
 segment  t_start  t_end axis  ref_final  step_amp  overshoot_pct  settling_time_s  steady_state_error     rmse
       0      0.0  2.998    x        0.0  0.000000         0.0000              NaN            0.000000 0.000000
       0      0.0  2.998    y        0.0  0.000000         0.0000              NaN            0.000000 0.000000
       0      0.0  2.998    z        1.0  1.000000         6.3013            2.458            0.049639 0.388644
       1      3.0 10.000    x        1.0  1.000000         4.2484            1.772           -0.000035 0.321173
       1      3.0 10.000    y        0.0  0.000000         0.0000              NaN            0.000000 0.000000
       1      3.0 10.000    z        1.0 -0.025162         0.0000            2.158           -0.000006 0.003389

Saved: csv/metrics_summary.csv
